In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

## 이미지에 바운딩 박스 그리고 저장 함수

In [2]:
import os
import cv2
from ultralytics import YOLO

def draw_bounding_boxes(image, predictions):
    """탐지된 핸드폰 객체에 바운딩 박스를 그림."""
    for prediction in predictions:
        x1, y1, x2, y2, conf, class_id = prediction
        if int(class_id) == 0:  # 핸드폰 클래스 ID가 0일 경우
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(image, f"Phone: {conf:.2f}", (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return image

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
def process_images_adj(input_folder, output_folder, model_path, confidence_threshold=0.3, iou_threshold=0.5):
    """입력 폴더 내 모든 이미지를 탐지하여 출력 폴더에 저장."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    model = YOLO(model_path)  # 모델은 루프 바깥에서 한번만 로드

    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.lower().endswith(".jpg"):
                input_path = os.path.join(root, file)

                # 상대경로 유지해서 저장 (하위폴더 포함)
                relative_path = os.path.relpath(input_path, input_folder)
                output_path = os.path.join(output_folder, relative_path)
                os.makedirs(os.path.dirname(output_path), exist_ok=True)

                image = cv2.imread(input_path)
                if image is None:
                    print(f"이미지 로드 실패: {input_path}")
                    continue

                model = YOLO(model_path)
                results = model(image, conf=confidence_threshold, iou=iou_threshold)
                predictions = results[0].boxes.data

                image_with_boxes = draw_bounding_boxes(image, predictions)

                cv2.imwrite(output_path, image_with_boxes)
                print(f"저장 완료: {output_path}")

## 실행

In [8]:
model_path = '/content/drive/MyDrive/phone_finetunning_0415/exp12/weights/best.pt'

data_path = "/content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/PhoneCall"
save_path = '/content/drive/MyDrive/DMS/phone_BB/phonecall_under70_0428'
folder_name = ['gF-22(File21)_s2_face_2019-04-09-11;41;13_22_1.avi',
                'gA-5(File5)_s4_face_2019-03-22-11;28;00_5_2.avi',
                'gZ-36(File30)_s4_face_2019-04-03-15;01;29_36_3.avi',
                'gE-28(File18)_s2_face_2019-03-15-10;12;30_28_0.avi',
                'gA-4(File4)_s2_face_2019-03-13-10;43;06_4_0.avi',
                'gE-28(File18)_s2_face_2019-03-15-10;12;30_28_1.avi',
                'gB-7(File7)_s4_face_2019-03-22-10;42;09_7_0.avi',
                'gF-24(File23)_s4_face_2019-03-25-14;55;37_24_0.avi',
                'gZ-36(File30)_s4_face_2019-04-03-15;01;29_36_2.avi',
                'gF-25(File24)_s4_face_2019-03-20-10;29;08_25_5.avi']
save_folder_name = folder_name

for i in folder_name:
    input_folder = os.path.join(data_path, i)
    output_folder = os.path.join(save_path, i)
    process_images_adj(input_folder, output_folder, model_path)


0: 224x224 (no detections), 429.9ms
Speed: 0.8ms preprocess, 429.9ms inference, 0.8ms postprocess per image at shape (1, 3, 224, 224)
저장 완료: /content/drive/MyDrive/DMS/phone_BB/phonecall_under70_0428/gF-22(File21)_s2_face_2019-04-09-11;41;13_22_1.avi/frame000000.jpg

0: 224x224 (no detections), 271.7ms
Speed: 1.2ms preprocess, 271.7ms inference, 0.8ms postprocess per image at shape (1, 3, 224, 224)
저장 완료: /content/drive/MyDrive/DMS/phone_BB/phonecall_under70_0428/gF-22(File21)_s2_face_2019-04-09-11;41;13_22_1.avi/frame000001.jpg

0: 224x224 (no detections), 233.3ms
Speed: 1.1ms preprocess, 233.3ms inference, 0.6ms postprocess per image at shape (1, 3, 224, 224)
저장 완료: /content/drive/MyDrive/DMS/phone_BB/phonecall_under70_0428/gF-22(File21)_s2_face_2019-04-09-11;41;13_22_1.avi/frame000002.jpg

0: 224x224 (no detections), 332.4ms
Speed: 0.7ms preprocess, 332.4ms inference, 0.7ms postprocess per image at shape (1, 3, 224, 224)
저장 완료: /content/drive/MyDrive/DMS/phone_BB/phonecall_under70_